# Sprint 3 · Webinar 8 — Clase Práctica de SQL (DuckDB en navegador)

**Enfoque:** practicar SQL desde cero usando DuckDB en https://sql-workbench.com/ (sin Python).

**Duración sugerida:** 100 min

---

**Dataset (CSV en GitHub RAW):**
- customers: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.csv
- order_items: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.csv
- orders: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.csv
- payments: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.csv
- products: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.csv


## Objetivos de aprendizaje

Al finalizar la sesión, podrás:

1. Cargar datasets CSV desde GitHub (RAW) en DuckDB usando `read_csv_auto()`.
2. Formular consultas con `SELECT`, `WHERE`, `ORDER BY` y `LIMIT`.
3. Unir tablas con `JOIN` para responder preguntas de negocio.
4. Construir KPIs con `GROUP BY` y agregaciones (`COUNT`, `SUM`, `AVG`).
5. Usar `HAVING` y `CASE` para filtrar y segmentar resultados.


## Agenda (100 min)

- 0) Configuración y carga del dataset (10 min)
- 1) Sanity checks y exploración rápida (5 min)
- 2) Ejercicio 0 — Calentamiento (Breakout Rooms) (10 min)
- 3) Ejercicios guiados 1–5 (65 min)
- 4) Retos + cierre (10 min)


## 0) Configuración rápida en DuckDB (Workbench)

1. Abre https://sql-workbench.com/
2. Selecciona **DuckDB** (SQL Editor).
3. Pega el bloque de carga de datos y ejecútalo.

> **Importante (alineado con el Teórico):**  
> En esta sesión **NO usamos** CTE (`WITH ...`) ni **window functions**.  
> Nos enfocamos en: `SELECT`, `WHERE`, `ORDER BY`, `LIMIT`, `JOIN`, `GROUP BY`, agregaciones y `HAVING` + `CASE` básico.


```sql
-- 0) Abre DuckDB en el navegador: https://sql-workbench.com/
--    (elige DuckDB / SQL Editor)

-- 1) Carga el dataset desde GitHub como VIEWS (no necesitas CREATE TABLE)
CREATE OR REPLACE VIEW customers AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.csv');

CREATE OR REPLACE VIEW products AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.csv');

CREATE OR REPLACE VIEW orders AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.csv');

CREATE OR REPLACE VIEW order_items AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.csv');

CREATE OR REPLACE VIEW payments AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.csv');
```


## 1) Verificación rápida (Sanity checks)

Ejecuta estas consultas para confirmar que todo cargó bien.

```sql
-- Sanity checks (2 min)
SELECT COUNT(*) AS n_customers FROM customers;
SELECT COUNT(*) AS n_products  FROM products;
SELECT COUNT(*) AS n_orders    FROM orders;
SELECT COUNT(*) AS n_items     FROM order_items;
SELECT COUNT(*) AS n_payments  FROM payments;

-- Muestras rápidas
SELECT * FROM customers LIMIT 5;
SELECT * FROM products LIMIT 5;
SELECT * FROM orders LIMIT 5;
SELECT * FROM order_items LIMIT 5;
SELECT * FROM payments LIMIT 5;
```


## 2) Ejercicio 0 (Breakout Rooms)

### Ejercicio 0 — Calentamiento (Breakout Rooms) (10 min)

**Contexto:** queremos un mini-resumen del negocio para “romper el hielo”.

**Objetivo:** calcular, por **estado de la orden** y **método de pago**:
- número de órdenes
- total pagado (`SUM(amount_paid)`)

**Tablas:** `orders`, `payments`

**Pistas:**
1. Une `orders` con `payments` por `order_id`.
2. Agrupa por `orders.status` y `payments.payment_method`.
3. Usa `COUNT(DISTINCT orders.order_id)` para contar órdenes.
4. Ordena por `total_pagado` desc.

**Plantilla (rellena los huecos):**
```sql
SELECT
  o.status,
  p.payment_method,
  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.amount_paid)        AS total_pagado
FROM orders o
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY
  o.status,
  p.payment_method
ORDER BY total_pagado DESC;
```

✅ **Solución:** es la misma consulta de arriba (si te salió, perfecto).


## 3) Ejercicios guiados

> En cada ejercicio: lee el objetivo → usa las pistas → intenta con la plantilla → compara con la solución.


### Ejercicio 1 — Órdenes por segmento de cliente (12 min)

**Objetivo:** identificar qué tipo de clientes compra más.

**Qué debes obtener (por `segment`):**
- número de órdenes (`n_orders`)
- total pagado (`total_pagado`)
- ticket promedio (`avg_ticket`)

**Tablas:** `customers`, `orders`, `payments`

**Pistas:**
- `customers` → `orders` por `customer_id`
- `orders` → `payments` por `order_id`
- `avg_ticket` = `SUM(amount_paid) / COUNT(DISTINCT order_id)`

**Guía (ejemplo de estructura):**
```sql
SELECT
  c.segment,
  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.amount_paid)         AS total_pagado,
  SUM(p.amount_paid) / COUNT(DISTINCT o.order_id) AS avg_ticket
FROM customers c
JOIN orders o
  ON c.customer_id = o.customer_id
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY c.segment
ORDER BY total_pagado DESC;
```

✅ **Solución:** igual a la consulta guía (si te dio error, revisa alias y nombres de columnas).


### Ejercicio 2 — Ingresos por categoría (GROUP BY + JOIN) (15 min)

**Objetivo:** calcular ingresos (revenue) por `category`.

**Cómo calcular revenue por ítem:**
- `revenue_item = quantity * unit_price * (1 - discount_rate)`

**Tablas:** `order_items`, `products`

**Pistas:**
- Une `order_items` con `products` por `product_id`.
- Agrupa por `products.category`.
- Usa `SUM(...)` para el revenue total.

**Plantilla:**
```sql
SELECT
  pr.category,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue
FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY revenue DESC;
```

✅ **Solución:** la misma plantilla.


### Ejercicio 3 — Top 5 productos por revenue (ORDER BY + LIMIT) (12 min)

**Objetivo:** encontrar los 5 productos que más dinero generan.

**Qué debes mostrar:**
- `product_id`, `title`, `category`
- revenue total del producto

**Pistas:**
- Une `order_items` con `products`.
- Agrupa por `product_id`, `title`, `category`.
- `ORDER BY revenue DESC` y `LIMIT 5`.

**Solución (consulta completa):**
```sql
SELECT
  pr.product_id,
  pr.title,
  pr.category,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue
FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY
  pr.product_id,
  pr.title,
  pr.category
ORDER BY revenue DESC
LIMIT 5;
```


### Ejercicio 4 — Margen bruto por categoría (20 min)

**Objetivo:** estimar rentabilidad por categoría.

**Cálculos:**
- `revenue = SUM(quantity * unit_price * (1 - discount_rate))`
- `cost = SUM(quantity * cost)`  *(cost viene de `products`)*
- `gross_profit = revenue - cost`
- `margin_pct = 100 * gross_profit / revenue`

**Tablas:** `order_items`, `products`

**Pistas:**
- En SQL, para evitar división por cero, puedes usar:
  - `CASE WHEN revenue = 0 THEN NULL ELSE ... END`
- Usa alias para mejorar legibilidad.

**Guía + Solución:**
```sql
SELECT
  pr.category,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue,
  SUM(oi.quantity * pr.cost)                                AS cost,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) - SUM(oi.quantity * pr.cost) AS gross_profit,
  CASE
    WHEN SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) = 0 THEN NULL
    ELSE 100 * (
      (SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) - SUM(oi.quantity * pr.cost))
      / SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate))
    )
  END AS margin_pct
FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY revenue DESC;
```


### Ejercicio 5 — ¿Las promos sirven? (CASE + GROUP BY + HAVING) (18 min)

**Objetivo:** comparar órdenes **con promo** vs **sin promo**.

**Qué debes obtener por grupo:**
- `grupo_promo`: 'con_promo' / 'sin_promo'
- `n_orders`
- `total_descuento` (desde `payments.discount`)
- `total_pagado` (desde `payments.amount_paid`)
- `avg_ticket`

**Tablas:** `orders`, `payments`

**Pistas:**
- `promo_code` puede venir vacío o nulo (trátalo como “sin promo”).
- Usa `CASE WHEN ... THEN ... ELSE ... END`.
- Agrega un `HAVING` para quedarte solo con grupos con al menos 50 órdenes (si aplica).

**Solución:**
```sql
SELECT
  CASE
    WHEN o.promo_code IS NULL OR o.promo_code = '' THEN 'sin_promo'
    ELSE 'con_promo'
  END AS grupo_promo,
  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.discount)            AS total_descuento,
  SUM(p.amount_paid)         AS total_pagado,
  SUM(p.amount_paid) / COUNT(DISTINCT o.order_id) AS avg_ticket
FROM orders o
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY grupo_promo
-- Si en tu dataset algún grupo queda muy pequeño, comenta HAVING.
HAVING COUNT(DISTINCT o.order_id) >= 50
ORDER BY total_pagado DESC;
```


## 7) Retos (opcional, para casa)

1) **Clientes frecuentes**  
Encuentra los `customer_id` con **3 o más órdenes** y muestra:
- `customer_id`, `customer_name`, `n_orders`, `total_pagado`

Pista: `customers` + `orders` + `payments`, `GROUP BY customer`.

2) **Productos con descuento alto**  
Lista productos (title) donde el descuento promedio (`AVG(discount_rate)`) sea **>= 0.20**.

Pista: `order_items` + `products`, `GROUP BY product`, `HAVING AVG(...)`.

3) **Categorías “pequeñas” pero rentables**  
Muestra categorías con revenue total **< 5,000** pero margen **> 25%**.

Pista: usa la lógica del Ejercicio 4 + `HAVING` con dos condiciones.


## Cierre (5 min)

**Checklist de lo que ya puedes hacer:**
- Cargar datos desde un CSV remoto (`read_csv_auto`) y trabajar como si fueran tablas
- Resolver preguntas de negocio con `JOIN` + `GROUP BY`
- Calcular KPIs: revenue, costo, profit, ticket promedio
- Usar `HAVING` para filtrar agregaciones y `CASE` para segmentar

**Siguiente paso recomendado:**  
Repite los ejercicios cambiando una condición (por fecha, por status, por método de pago) y verifica cómo cambian los KPIs.


## Siguientes pasos

- Practica cambiando **una sola variable** por ejercicio (status, categoría, método de pago, etc.).
- Como preparación para el proyecto: intenta construir un **mini-reporte** con 4 KPIs del negocio (ventas, margen, ticket, top productos).
